In [45]:
import os
import re
import glob
import datetime
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import rcParams
import netCDF4
import logging
#from plot_AVISO_data import prepareDateStrings

## Configuration

Logging

In [46]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

Plotting (should be ideally in `matplotlibrc` but seems not to work with virtualenv).

In [47]:
rcParams.update({'font.size': 12, 'figure.dpi': 300, 'savefig.bbox': 'tight'})
# rcParams.keys

# Input parameters

## Files and directories

In [48]:
databasedir = "/home/ctroupin/Data/AVISO/regional-mediterranean/delayed-time/along-track/filtered/adt"
resultdir = "/home/ctroupin/Projects/201704_DivaAltimetry/results/divand"
figdir = "/home/ctroupin/Projects/201704_DivaAltimetry/figures/divand/"
resultfile = os.path.join(resultdir, "ADT_20140515_20140604.nc")

## Domain and period of interest

In [49]:
coordinates = [-6.75, 40.001, 30, 48.]
year, month, day, interval = 2014, 5, 25, 10
timescale = 5.        # temporal scale in days

## Interpolation grid

In [96]:
lonmin, lonmax, latmin, latmax = -7., 40., 30., 48.
dx, dy, dt = 0.25, 0.25, 1.
lonr = np.arange(lonmin, lonmax + .1, dx)
latr = np.arange(latmin, latmax +.1, dy)
timer = np.arange(23510, 23529 + .1, dt)

## Map projection

In [86]:
m = Basemap(projection='merc',
            llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='i')

# Make the plots

## Read netCDF file

File not too big so I read all its content at the same time.<br>
If needed easy to read slice by slice in the loop.

In [99]:
nc = netCDF4.Dataset(resultfile)
field = nc.variables['ADT'][:]
nc.close()
field.shape

(20, 73, 189)

## Loop on the time

In [100]:
llonr, llatr = np.meshgrid(lonr, latr)
# Loop on time
for tt, ttime in enumerate(timer):
    
    fig = plt.figure()
    m.pcolormesh(llonr, llatr, field[tt, :, :], latlon=True, vmin=-.2, vmax=.2)
    # Add lines, coastline etc    
    m.drawmeridians(np.arange(-8., round(coordinates[1]), 8), 
                    labels=[0, 0, 0, 1], linewidth=.2)
    m.drawparallels(np.linspace(round(coordinates[2]), round(coordinates[3]), 5), 
                    labels=[1, 0, 0, 0], linewidth=.2)
    m.drawcoastlines(linewidth=.2)
    m.fillcontinents()
    cbar = plt.colorbar(shrink=.6, extend='both')
    cbar.set_label(" m", rotation=0)
    
    dateref = datetime.datetime(1950, 1, 1).toordinal()
    datefield = datetime.datetime.fromordinal(int(timer[tt] + dateref))
    figtitledate = datefield.strftime("%Y-%m-%d")
    fignamedate = datefield.strftime("%Y%m%d")
    
    plt.title(figtitledate)
    plt.savefig(os.path.join(figdir, "ADT_divand_{0}".format(fignamedate)), dpi=300)
    #plt.show()
    plt.close()


/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3413: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3422: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)
/home/ctroupin/Software/PythonEnvs/Diva-python3.6/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1767: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  axisbgc = ax.get_axis_bgcolor()
